**Import library**

In [39]:
from openai import AzureOpenAI
import pyodbc
import pandas as pd

import os
from dotenv import load_dotenv
from pathlib import Path

import re
from datetime import datetime

**Locate env path**

In [40]:
script_dir = Path(os.getcwd())
env_path = script_dir / '.env'
load_dotenv(env_path)

print(f'Script Path:\t{script_dir}\nenv Path:\t{env_path}\nLoad env:\t{load_dotenv(env_path)}')

Script Path:	c:\Users\Admins\Desktop\Tae\work\azure-sql-2025-tester
env Path:	c:\Users\Admins\Desktop\Tae\work\azure-sql-2025-tester\.env
Load env:	True


In [41]:
server = os.getenv('azure_server')
database = os.getenv('azure_database')
username = os.getenv('azure_username')
password = os.getenv('azure_password')

**Connection Database**

In [42]:
conn_str = f"""Driver={{ODBC Driver 17 for SQL Server}}; Server={server}; Database={database}; Uid={username}; Pwd={password}; Encrypt=yes; TrustServerCertificate=no; Connection Timeout=30;"""

In [43]:
conn = pyodbc.connect(conn_str)
conn

**Ai Prompt**

In [44]:
# Path to your Schema.sql file
sql_file_path = script_dir / "sql_script" / "Schema.sql"

# Read the file content into a Python string
with open(sql_file_path, 'r', encoding='utf-8') as file:
    schema_query = file.read()

print(schema_query)

-- =============================================
-- Author:		TANAKRIT-THONGPENG
-- Create date: 27/06/2025
-- Description:	SCHEMA QUERY
-- =============================================

SELECT 
    TABLE_CATALOG, 
    TABLE_SCHEMA, 
    TABLE_NAME, 
    COLUMN_NAME, 
    DATA_TYPE
FROM 
    INFORMATION_SCHEMA.COLUMNS
ORDER BY 
    TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;


In [45]:
database = 'mydemo2025'
schema = 'SalesLT'

schema_df = pd.read_sql_query(schema_query, conn)
schema_df = schema_df[schema_df['TABLE_CATALOG'] == f'{database}']
schema_df = schema_df[schema_df['TABLE_SCHEMA'] == f'{schema}']

schema_text = ""
for (schema, table), group in schema_df.groupby(['TABLE_SCHEMA', 'TABLE_NAME']):
    columns = ", ".join(f"{row.COLUMN_NAME} ({row.DATA_TYPE})" for _, row in group.iterrows())
    schema_text += f"Table {schema}.{table}: {columns}\n"


C:\Users\Admins\AppData\Local\Temp\ipykernel_20472\1975608560.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql_query(schema_query, conn)


In [46]:
schema_df

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE
13,mydemo2025,SalesLT,Address,AddressID,int
14,mydemo2025,SalesLT,Address,AddressLine1,nvarchar
15,mydemo2025,SalesLT,Address,AddressLine2,nvarchar
16,mydemo2025,SalesLT,Address,City,nvarchar
17,mydemo2025,SalesLT,Address,StateProvince,nvarchar
...,...,...,...,...,...
137,mydemo2025,SalesLT,vProductModelCatalogDescription,ProductLine,nvarchar
138,mydemo2025,SalesLT,vProductModelCatalogDescription,Style,nvarchar
139,mydemo2025,SalesLT,vProductModelCatalogDescription,RiderExperience,nvarchar
140,mydemo2025,SalesLT,vProductModelCatalogDescription,rowguid,uniqueidentifier


In [47]:
user_prompt = "Can you add more table, The table should have at least 1 field that keeping json format as a nvarchar and should be related to the existing tables in the schema."

In [48]:
model_name = os.getenv("openai_model_name")

client = AzureOpenAI(
    api_key=os.getenv("openai_model_api_key"),
    api_version=os.getenv("openai_api_version"),
    azure_endpoint=os.getenv("openai_api_endpoint")
)

In [49]:
system_message = f"""
You are a helpful assistant that writes SQL Server queries.
Here is the database schema:

{schema_text}
"""

In [50]:
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"Write a SQL Server query for this: {user_prompt}"}
    ]
)

**Generate Output**

In [51]:
generated_sql = response.choices[0].message.content
print("Generated SQL:\n", generated_sql)

Generated SQL:
 Certainly! Here is an example of adding a new table called "ProductReview" with a field named "ReviewInfo" that stores data in JSON format as an NVARCHAR, and establishing a relationship with the existing "SalesLT.Product" table:

```sql
-- Create the ProductReview table
CREATE TABLE ProductReview (
    ReviewID INT PRIMARY KEY,
    ProductID INT,
    ReviewInfo NVARCHAR(MAX),
    CONSTRAINT FK_ProductReview_ProductID FOREIGN KEY (ProductID) REFERENCES SalesLT.Product(ProductID)
);

-- Insert a sample record with JSON data in the ReviewInfo field
INSERT INTO ProductReview (ReviewID, ProductID, ReviewInfo)
VALUES (1, 1, '{"Rating": 5, "Comment": "Great product, highly recommended"}');

-- Query to retrieve Product information along with related Product Review
SELECT p.ProductID, p.Name, pr.ReviewInfo
FROM SalesLT.Product p
LEFT JOIN ProductReview pr ON p.ProductID = pr.ProductID;
```

In this example, the "ProductReview" table is created with a foreign key relationship t

In [52]:
# Extract just the SQL code between ```sql and ```
sql_code_blocks = re.findall(r"```sql\s*(.*?)```", response.choices[0].message.content, re.DOTALL)
description_blocks = re.split(r"```(?:sql)?[\s\S]*?```", response.choices[0].message.content)

if sql_code_blocks:
    generated_sql = sql_code_blocks[0].strip()
    generated_description = description_blocks[1].strip()
else:
    generated_sql = response.choices[0].message.content.strip()  # fallback
    generated_description = response.choices[0].message.content.strip()  # fallback

print(f"{generated_sql}\n\n{generated_description}")

-- Create the ProductReview table
CREATE TABLE ProductReview (
    ReviewID INT PRIMARY KEY,
    ProductID INT,
    ReviewInfo NVARCHAR(MAX),
    CONSTRAINT FK_ProductReview_ProductID FOREIGN KEY (ProductID) REFERENCES SalesLT.Product(ProductID)
);

-- Insert a sample record with JSON data in the ReviewInfo field
INSERT INTO ProductReview (ReviewID, ProductID, ReviewInfo)
VALUES (1, 1, '{"Rating": 5, "Comment": "Great product, highly recommended"}');

-- Query to retrieve Product information along with related Product Review
SELECT p.ProductID, p.Name, pr.ReviewInfo
FROM SalesLT.Product p
LEFT JOIN ProductReview pr ON p.ProductID = pr.ProductID;

In this example, the "ProductReview" table is created with a foreign key relationship to the "SalesLT.Product" table using the "ProductID" field. The "ReviewInfo" field in the "ProductReview" table stores JSON data related to product reviews.


In [53]:
query_string = f"""{generated_sql}"""
query_string

'-- Create the ProductReview table\nCREATE TABLE ProductReview (\n    ReviewID INT PRIMARY KEY,\n    ProductID INT,\n    ReviewInfo NVARCHAR(MAX),\n    CONSTRAINT FK_ProductReview_ProductID FOREIGN KEY (ProductID) REFERENCES SalesLT.Product(ProductID)\n);\n\n-- Insert a sample record with JSON data in the ReviewInfo field\nINSERT INTO ProductReview (ReviewID, ProductID, ReviewInfo)\nVALUES (1, 1, \'{"Rating": 5, "Comment": "Great product, highly recommended"}\');\n\n-- Query to retrieve Product information along with related Product Review\nSELECT p.ProductID, p.Name, pr.ReviewInfo\nFROM SalesLT.Product p\nLEFT JOIN ProductReview pr ON p.ProductID = pr.ProductID;'

**Final Dataframe**

In [54]:
try:
    df = pd.read_sql_query(query_string, conn)
    print("Query Results:")
    display(df)
except Exception as e:
    print("Error running the query:", e)

Error running the query: Execution failed on sql '-- Create the ProductReview table
CREATE TABLE ProductReview (
    ReviewID INT PRIMARY KEY,
    ProductID INT,
    ReviewInfo NVARCHAR(MAX),
    CONSTRAINT FK_ProductReview_ProductID FOREIGN KEY (ProductID) REFERENCES SalesLT.Product(ProductID)
);

-- Insert a sample record with JSON data in the ReviewInfo field
INSERT INTO ProductReview (ReviewID, ProductID, ReviewInfo)
VALUES (1, 1, '{"Rating": 5, "Comment": "Great product, highly recommended"}');

-- Query to retrieve Product information along with related Product Review
SELECT p.ProductID, p.Name, pr.ReviewInfo
FROM SalesLT.Product p
LEFT JOIN ProductReview pr ON p.ProductID = pr.ProductID;': ('23000', '[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK_ProductReview_ProductID". The conflict occurred in database "mydemo2025", table "SalesLT.Product", column \'ProductID\'. (547) (SQLExecDirectW); [23000] 

C:\Users\Admins\AppData\Local\Temp\ipykernel_20472\1492314482.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_string, conn)


**Create SQL Script from Output**

In [55]:
Author = "AI Generated SQL Query"

day = datetime.now().day
month = datetime.now().month
year = datetime.now().year
date_obj = datetime(year, month, day)
create_date = date_obj.strftime("%d/%m/%Y")

description = generated_description


In [56]:
header = f"""-- =============================================
-- Author:		{Author}
-- Create date: {create_date}
-- Description:	{description}
-- =============================================
"""

In [57]:
header

'-- =============================================\n-- Author:\t\tAI Generated SQL Query\n-- Create date: 30/06/2025\n-- Description:\tIn this example, the "ProductReview" table is created with a foreign key relationship to the "SalesLT.Product" table using the "ProductID" field. The "ReviewInfo" field in the "ProductReview" table stores JSON data related to product reviews.\n-- =============================================\n'

In [58]:
query_string

'-- Create the ProductReview table\nCREATE TABLE ProductReview (\n    ReviewID INT PRIMARY KEY,\n    ProductID INT,\n    ReviewInfo NVARCHAR(MAX),\n    CONSTRAINT FK_ProductReview_ProductID FOREIGN KEY (ProductID) REFERENCES SalesLT.Product(ProductID)\n);\n\n-- Insert a sample record with JSON data in the ReviewInfo field\nINSERT INTO ProductReview (ReviewID, ProductID, ReviewInfo)\nVALUES (1, 1, \'{"Rating": 5, "Comment": "Great product, highly recommended"}\');\n\n-- Query to retrieve Product information along with related Product Review\nSELECT p.ProductID, p.Name, pr.ReviewInfo\nFROM SalesLT.Product p\nLEFT JOIN ProductReview pr ON p.ProductID = pr.ProductID;'

In [59]:
def find_latest_script_number(folder_path):
    folder = Path(folder_path)
    max_number = 0

    # Pattern to match filenames like Script-<number>.sql
    pattern = re.compile(r"Script-(\d+)\.sql", re.IGNORECASE)

    for file in folder.glob("Script-*.sql"):
        match = pattern.match(file.name)
        if match:
            number = int(match.group(1))
            if number > max_number:
                max_number = number

    return max_number

In [60]:
folder_path = script_dir / "sql_script"
latest_number = find_latest_script_number(folder_path)
print("Latest script number:", latest_number)

Latest script number: 2


In [61]:
output_path = script_dir / "sql_script" / f"Script-{latest_number+1}.sql"

In [62]:
# Make sure the directory exists, create if not
output_path.parent.mkdir(parents=True, exist_ok=True)

In [63]:
full_sql = header + "\n" + query_string

# Write to file
with open(output_path, "w", encoding="utf-8") as f:
    f.write(full_sql)

print(f"SQL file saved to: {output_path}")

SQL file saved to: c:\Users\Admins\Desktop\Tae\work\azure-sql-2025-tester\sql_script\Script-3.sql
